In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from fhir_client import FHIRClient
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from preprocessing import Preprocessing

preprocessor = Preprocessing()
client = FHIRClient(service_base_url='https://r3.smarthealthit.org', preprocessor=preprocessor, logger=logger)

INFO:root:Processor ObservationLatestBmiProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestBmiProcessor
INFO:root:Name of patient processor will be PatientbmiLatestProcessor
INFO:root:Processor PatientbmiLatestProcessor added.
INFO:root:Processor ObservationLatestHeightProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestHeightProcessor
INFO:root:Name of patient processor will be PatientheightLatestProcessor
INFO:root:Processor PatientheightLatestProcessor added.
INFO:root:Processor ObservationLatestWeightProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestWeightProcessor
INFO:root:Name of patient processor will be PatientweightLatestProcessor
INFO:root:Processor PatientweightLatestProcessor added.
INFO:__main__:Capability statement of https://r3.smarthealthit.org was successfully received.


In [3]:
client.preprocessor.registered_observation_processors

{'ObservationLatestBmiProcessor': preprocessing.Preprocessing.ObservationLatestBmiProcessor,
 'ObservationLatestHeightProcessor': preprocessing.Preprocessing.ObservationLatestHeightProcessor,
 'ObservationLatestWeightProcessor': preprocessing.Preprocessing.ObservationLatestWeightProcessor}

In [9]:
procedures = client.get_all_procedures()
pd.DataFrame([prod.code['coding'][0] for prod in procedures]).drop_duplicates().sort_values(by=['display']).head()

INFO:root:Received 18117 procedures in 60.80 seconds.


,code,display,system
893,183450002,Admission to burn unit,http://snomed.info/sct
1911,305340004,Admission to long stay hospital,http://snomed.info/sct
83,305428000,Admission to orthopedic department,http://snomed.info/sct
6217,305433001,Admission to trauma surgery department,http://snomed.info/sct
13687,35637008,Alcohol rehabilitation,http://snomed.info/sct


In [ ]:
conditions = client.get_all_conditions()
pd.DataFrame([cond.code['coding'][0] for cond in conditions]).drop_duplicates().sort_values(by=['display']).head()

In [19]:
# Get patients by condition
patients_by_condition_text = client.get_patients_by_condition_text("Acute viral pharyngitis")

INFO:root:Received 43 observations in 0.24 seconds.
INFO:root:Received 41 observations in 0.37 seconds.
INFO:root:Received 21 observations in 0.19 seconds.
INFO:root:Received 16 observations in 0.22 seconds.
INFO:root:Received 181 observations in 1.12 seconds.
INFO:root:Received 36 observations in 0.30 seconds.
INFO:root:Received 49 observations in 0.31 seconds.
INFO:root:Received 21 observations in 0.20 seconds.
INFO:root:Received 139 observations in 0.72 seconds.
INFO:root:Received 58 observations in 0.43 seconds.
INFO:root:Received 135 observations in 0.53 seconds.
INFO:root:Received 42 observations in 0.27 seconds.
INFO:root:Received 29 observations in 0.31 seconds.
INFO:root:Received 118 observations in 0.58 seconds.
INFO:root:Received 41 observations in 0.33 seconds.
INFO:root:Received 43 observations in 0.55 seconds.
INFO:root:Received 23 observations in 0.17 seconds.
INFO:root:Received 53 observations in 0.49 seconds.
INFO:root:Received 56 observations in 0.44 seconds.
INFO:roo

INFO:root:Received 29 observations in 0.19 seconds.
INFO:root:Received 204 observations in 0.77 seconds.
INFO:root:Received 63 observations in 0.40 seconds.
INFO:root:Received 34 observations in 0.19 seconds.
INFO:root:Received 67 observations in 0.43 seconds.
INFO:root:Received 31 observations in 0.18 seconds.
INFO:root:Received 53 observations in 0.32 seconds.
INFO:root:Received 153 observations in 0.65 seconds.
INFO:root:Received 25 observations in 0.17 seconds.
INFO:root:Received 22 observations in 0.17 seconds.
INFO:root:Received 86 observations in 0.36 seconds.
INFO:root:Received 42 observations in 0.20 seconds.
INFO:root:Received 154 observations in 0.68 seconds.
INFO:root:Received 20 observations in 0.19 seconds.
INFO:root:Received 71 observations in 0.34 seconds.
INFO:root:Received 34 observations in 0.19 seconds.
INFO:root:Received 58 observations in 0.32 seconds.
INFO:root:Received 55 observations in 0.42 seconds.
INFO:root:Received 38 observations in 0.26 seconds.
INFO:root

INFO:root:Received 95 observations in 0.45 seconds.
INFO:root:Received 41 observations in 0.19 seconds.
INFO:root:Received 30 observations in 0.18 seconds.
INFO:root:Received 32 observations in 0.18 seconds.
INFO:root:Received 30 observations in 0.22 seconds.
INFO:root:Received 52 observations in 0.31 seconds.
INFO:root:Received 35 observations in 0.18 seconds.
INFO:root:Received 31 observations in 0.64 seconds.
INFO:root:Received 29 observations in 0.17 seconds.
INFO:root:Received 49 observations in 0.21 seconds.
INFO:root:Received 60 observations in 0.31 seconds.
INFO:root:Received 21 observations in 0.31 seconds.
INFO:root:Received 163 observations in 0.64 seconds.
INFO:root:Received 48 observations in 0.28 seconds.
INFO:root:Received 74 observations in 0.33 seconds.
INFO:root:Received 56 observations in 0.40 seconds.
INFO:root:Received 41 observations in 0.21 seconds.
INFO:root:Received 130 observations in 0.49 seconds.
INFO:root:Received 73 observations in 0.54 seconds.
INFO:root:

INFO:root:Received 28 observations in 0.20 seconds.
INFO:root:Received 73 observations in 0.61 seconds.
INFO:root:Received 43 observations in 0.31 seconds.
INFO:root:Received 74 observations in 0.30 seconds.
INFO:root:Received 139 observations in 0.62 seconds.
INFO:root:Received 21 observations in 0.16 seconds.
INFO:root:Received 30 observations in 0.15 seconds.
INFO:root:Received 49 observations in 0.19 seconds.
INFO:root:Received 54 observations in 0.30 seconds.
INFO:root:Received 52 observations in 0.52 seconds.
INFO:root:Received 49 observations in 0.20 seconds.
INFO:root:Received 33 observations in 0.19 seconds.
INFO:root:Received 17 observations in 0.16 seconds.
INFO:root:Received 69 observations in 0.31 seconds.
INFO:root:Received 104 observations in 0.54 seconds.
INFO:root:Received 61 observations in 0.44 seconds.
INFO:root:Received 29 observations in 0.21 seconds.
INFO:root:Received 41 observations in 0.20 seconds.
INFO:root:Received 39 observations in 0.18 seconds.
INFO:root:

In [3]:
patients_by_condition_text = client.get_patients_by_condition_text("Abdominal pain")

INFO:root:Received 154 observations in 0.67 seconds.
INFO:root:Received 59 observations in 0.53 seconds.
INFO:root:Received 162 observations in 0.67 seconds.
INFO:root:Received 206 observations in 1.05 seconds.
INFO:root:Received 4 patients in 3.10 seconds.


## Machine Learning

In [5]:
from fhir_objects.patient import Patient
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.svm import SVC

ml_fhir = MLOnFHIRClassifier(Patient, feature_attrs=['bmiLatest'], label_attrs=['gender'], random_state=44, preprocessor=client.preprocessor)
X, y, trained_clf = ml_fhir.fit(patients_by_condition_text, DecisionTreeClassifier())

from sklearn.metrics import accuracy_score, roc_curve, auc
fpr, tpr, _ = roc_curve(y, trained_clf.predict(X))
print("Prediction accuracy {}".format( auc(fpr, tpr) ) )

INFO:root:Extracting attributes from data set
INFO:root:Preprocessing data
INFO:root:Started training of clf
INFO:root:Training completed
INFO:root:Accuracy : 1.0, F1-score : 1.0


Prediction accuracy 1.0


In [6]:
# Evaluate classifier:
ml_fhir.evaluate(X, y)

{'accuracy': 1.0,
 'f1_score': 1.0,
 'precision': array([1., 1.]),
 'recall': array([1., 1.]),
 'f1_score_class': array([1., 1.]),
 'support': array([2, 2]),
 'AUROC': 1.0,
 'AUPRC': 1.0,
 'tn': 2,
 'fp': 0,
 'fn': 0,
 'tp': 2}

In [7]:
# Try clustering
from fhir_objects.patient import Patient
from ml_on_fhir import MLOnFHIRClassifier, MLOnFHIRCluster
from sklearn.cluster import KMeans

ml_fhir = MLOnFHIRCluster(Patient, feature_attrs=['birthDate'], label_attrs=['gender'], random_state=4, preprocessor=client.preprocessor)
X, y, trained_cluster = ml_fhir.fit(patients_by_condition_text, KMeans(n_clusters=2))

INFO:root:Extracting attributes from data set
INFO:root:Preprocessing data
INFO:root:Started clustering
INFO:root:Clustering completed
/Users/bockc/.local/share/virtualenvs/ml-on-fhir-Fls4lXwQ/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
INFO:root:Adjusted Rand index : 0.0, mutual information : 2.3256314468733853e-16, silhouette score : 0.6084558823529412


In [8]:
trained_cluster.score(X, y)

-68.66666666666606

# Experimental Section

In [21]:
%autoreload 2

import preprocessing

from sklearn.base import BaseEstimator
import datetime as dt
import numpy as np
from fhir_objects.fhir_resources import date_format

from fhir_client import FHIRClient
import logging
logging.basicConfig(level=logging.INFO)
import pandas as pd

from ml_on_fhir import MLOnFHIRClassifier
from fhir_objects.patient import Patient
from sklearn.tree import DecisionTreeClassifier


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
client = FHIRClient(service_base_url='https://r3.smarthealthit.org', logger=logger, preprocessor=None)

#patients_by_condition_text = client.get_patients_by_condition_code(system="http://snomed.info/sct", code='284549007')


INFO:root:Processor ObservationLatestBmiProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestBmiProcessor
INFO:root:Name of patient processor will be PatientbmiLatestProcessor
INFO:root:Processor PatientbmiLatestProcessor added.
INFO:root:Processor ObservationLatestHeightProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestHeightProcessor
INFO:root:Name of patient processor will be PatientheightLatestProcessor
INFO:root:Processor PatientheightLatestProcessor added.
INFO:root:Processor ObservationLatestWeightProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestWeightProcessor
INFO:root:Name of patient processor will be PatientweightLatestProcessor
INFO:root:Processor PatientweightLatestProcessor added.
INFO:__main__:Capability statement of https://r3.smarthealthit.org was successfully received.


In [23]:

ml_fhir = MLOnFHIRClassifier(Patient, feature_attrs=['weightLatest', 'heightLatest', 'bmiLatest'],
                             label_attrs=['gender'], preprocessor=client.preprocessor)
X, y, trained_clf = ml_fhir.fit(patients_by_condition_text, DecisionTreeClassifier())

INFO:root:Extracting attributes from data set
INFO:root:Preprocessing data
INFO:root:Started training of clf
INFO:root:Training completed
INFO:root:Accuracy : 1.0, F1-score : 1.0


In [5]:
%autoreload 2

from fhir_client import FHIRClient
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from sklearn.base import BaseEstimator
from ml_on_fhir import MLOnFHIRClassifier
from fhir_objects.patient import Patient
from sklearn.tree import DecisionTreeClassifier

from preprocessing import Preprocessing, AbstractObservationProcessor, get_coding_condition

#register_preprocessor(ObservationLatestHeightProcessor)
preprocessor = Preprocessing()

class ObservationLatestHeightProcessor(AbstractObservationProcessor):
    """
    Class to transform the FHIR observation resource with loinc code 8302-2 (body height)
    to be usable as patient feature.
    """
    def __init__(self):
        super().__init__('heightBest')

    def transform(self, X, **transform_params):
        condition = get_coding_condition([{'system': 'http://loinc.org', 'code': '8302-2'}])
        heights = list(filter(condition, X))
        heights = sorted(heights, reverse=True)
        if len(heights) >= 1:
            return self.patient_attribute_name, float(heights[0].valueQuantity['value'])
        else:
            return self.patient_attribute_name, 0.0

    def fit(self, X, y=None, **fit_params):
        return self
    
#preprocessor.register_preprocessor(ObservationLatestHeightProcessor)
client = FHIRClient(service_base_url='https://r3.smarthealthit.org', logger=logger, preprocessor=preprocessor)


INFO:root:Processor ObservationLatestBmiProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestBmiProcessor
INFO:root:Name of patient processor will be PatientbmiLatestProcessor
INFO:root:Processor PatientbmiLatestProcessor added.
INFO:root:Processor ObservationLatestHeightProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestHeightProcessor
INFO:root:Name of patient processor will be PatientheightLatestProcessor
INFO:root:Processor PatientheightLatestProcessor added.
INFO:root:Processor ObservationLatestWeightProcessor added.
INFO:root:Adding Patient Processor for ObservationLatestWeightProcessor
INFO:root:Name of patient processor will be PatientweightLatestProcessor
INFO:root:Processor PatientweightLatestProcessor added.
INFO:__main__:Capability statement of https://r3.smarthealthit.org was successfully received.


In [24]:
patients_by_condition_text = client.get_patients_by_condition_text("Abdominal pain")

ml_fhir = MLOnFHIRClassifier(Patient,
                             feature_attrs=['weightLatest', 'heightLatest', 'bmiLatest'],
                             label_attrs=['gender'],
                             preprocessor=preprocessor
                            )
X, y, trained_clf = ml_fhir.fit(patients_by_condition_text, DecisionTreeClassifier())

X

INFO:root:Received 154 observations in 0.68 seconds.
INFO:root:Received 59 observations in 0.35 seconds.
INFO:root:Received 162 observations in 0.89 seconds.
INFO:root:Received 206 observations in 0.85 seconds.
INFO:root:Received 4 patients in 2.94 seconds.
INFO:root:Extracting attributes from data set
INFO:root:Preprocessing data
INFO:root:Started training of clf
INFO:root:Training completed
INFO:root:Accuracy : 1.0, F1-score : 1.0


array([[  0.   , 132.334,  17.   ],
       [  0.   ,   0.   ,   0.   ],
       [  0.   , 158.242,  43.3  ],
       [  0.   , 175.26 ,  17.   ]])